In [58]:
# モジュールのインポート

import numpy as np
import sklearn
import keras
import copy
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.datasets import fetch_openml

In [47]:
# 画像にノイズを加える関数


def add_noise(image, percentage):
    noisy_image = copy.deepcopy(image)
    mask = np.random.random(image.shape) < percentage
    noise = np.random.randint(0, 256, size=image.shape)
    noise_float = noise / 255.0
    noisy_image[mask] = noise_float[mask]
    return noisy_image

In [59]:
# データセットのインポート

mnist = fetch_openml("mnist_784", as_frame=False, version=1, parser="auto")

In [ ]:
# データの調整

X, y = mnist.data, mnist.target

# 0~1の浮動小数に変換
X = X / 255.0

# yをone-hot-encoding
y = y.astype(int)
num_labels = len(np.unique(y))
one_hot_target = np.zeros((X.shape[0], num_labels))
one_hot_target[range(X.shape[0]), y] = 1

X_train, X_test, y_train, y_test = train_test_split(
    X[:3000], one_hot_target[:3000], test_size=0.3, random_state=42
)

# サイズの確認
print(X_train.shape)
print(y_train.shape)
print(y_train[0:2])

In [74]:
# 訓練データにノイズを加える
percentage = 0.10

X_train_noise = [add_noise(image, percentage) for image in X_train]
X_train_noise = np.array(X_train_noise)

# サイズの確認
print(X_train_noise.shape)
print(X_train.shape)

(2100, 784)
(2100, 784)


In [41]:
# Relu関数とSoftmax関数


class Relu:
    def __init__(self):
        self.mask = None

    def forward(self, x):
        if isinstance(x, np.ndarray):
            self.mask = x <= 0
            out = x.copy()
            out[self.mask] = 0
        else:
            out = np.maximum(0, x)
        return out

    def backward(self, dout):
        if isinstance(dout, np.ndarray):
            self.mask = dout <= 0
            dx = np.ones_like(dout)
            dx[self.mask] = 0
        else:
            dx = 1 if dout > 0 else 0
        return dx


class Softmax:
    def __init__(self):
        self.out = None

    def forward(self, x):
        exp_x = np.exp(x - np.max(x))
        sum_exp_x = np.sum(exp_x)
        out = exp_x / sum_exp_x
        self.out = out
        return np.clip(out, 1e-15, 1 - 1e-15)

    def backward(self, dout):
        dx = dout * (self.out * (1 - self.out))
        return dx

In [65]:
# ニューラルネットワーク
class Neural_Network:
    def __init__(self, hid_unit, learning_rate):
        self.hid_unit = hid_unit
        in_unit = 784
        out_unit = 10

        limit1 = np.sqrt(2.0 / in_unit)
        limit2 = np.sqrt(2.0 / self.hid_unit)
        self.weights1_2 = np.random.randn(in_unit, self.hid_unit) * limit1
        self.weights2_3 = np.random.randn(self.hid_unit, out_unit) * limit2

        self.bias1 = np.zeros((1, self.hid_unit))
        self.bias2 = np.zeros((1, out_unit))

        self.learning_rate = learning_rate
        self.relu = Relu()
        self.softmax = Softmax()

    # フィードフォワード
    def Feedforward(self, X):
        self.hid_layer = self.relu.forward(np.dot(X, self.weights1_2) + self.bias1)
        output = self.softmax.forward(
            np.dot(self.hid_layer, self.weights2_3) + self.bias2
        )
        return output

    # 誤差逆伝播法
    def Back_propagation(self, X, y):
        output = self.Feedforward(X)
        delta_output = (y - output) * self.softmax.backward(output)
        # delta_output = y - output
        delta_hidden = np.dot(delta_output, self.weights2_3.T) * self.relu.backward(
            self.hid_layer
        )

        # 重みとバイアスの更新
        self.weights2_3 -= self.learning_rate * np.dot(self.hid_layer.T, delta_output)
        self.weights1_2 -= self.learning_rate * np.dot(X.T, delta_hidden)
        self.bias2 -= self.learning_rate * np.sum(delta_output, axis=0, keepdims=True)
        self.bias1 -= self.learning_rate * np.sum(delta_hidden, axis=0, keepdims=True)

    # バッチを用いた学習
    def train(self, X_train, y_train, epochs, batch_size):
        for epoch in range(epochs):
            for i in range(0, len(X_train), batch_size):
                X_batch = X_train[i : i + batch_size]
                y_batch = y_train[i : i + batch_size]

                self.Back_propagation(X_batch, y_batch)

                # バッチごとの損失
                output = self.Feedforward(X_batch)
                output = np.clip(output, 1e-15, 1 - 1e-15)
                batch_loss = -np.sum(y_batch * np.log(output))
                print(f"Epoch {epoch}, Batch {i // batch_size}: Loss {batch_loss}")

In [ ]:
# バッチ学習

hid_unit = 50
learning_rate = 0.001
epochs = 200
batch_size = 40
model_batch = Neural_Network(hid_unit, learning_rate)

model_batch.train(X_train_noise, y_train, epochs, batch_size)

In [ ]:
# 学習

epochs = 2000
hid_unit = 40
learning_rate = 0.001

model = Neural_Network(hid_unit, learning_rate)

for epoch in range(epochs):
    model.Feedforward(X_train_noise)
    model.Back_propagation(X_train_noise, y_train)

# テストデータでモデルを評価
predictions = model.Feedforward(X_test)
accuracy = np.mean(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1))
print(f"テストデータの正解率: {accuracy * 100:.2f}%")